Criado por:
*   Eduardo Lemos (@esniq1)
*   Leandro Filho (@futpythontrader)



# Instalando as Bibliotecas e as Dependências

In [ ]:
# Instalando o Selenium
!pip install selenium

# Atualizando o Ubuntu para executar corretamento o apt-install
!apt-get update

# Instalando o ChromeDrive e Trazendo ele para a Pasta Local
!apt install chromium-chromedriver

!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 7.4 MB/s 
     |████████████████████████████████| 384 kB 56.0 MB/s 
     |████████████████████████████████| 140 kB 57.4 MB/s 
     |████████████████████████████████| 58 kB 6.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.c

# Configuração do Web-Driver

In [ ]:
# Utilizando o WebDriver do Selenium
from selenium import webdriver

# Instanciando o Objeto ChromeOptions
options = webdriver.ChromeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Chrome('chromedriver',options=options)

# Importando as Bibliotecas

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
from selenium.webdriver.common.by import By

# Iniciando a Raspagem de Dados

In [ ]:
# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com/")

In [ ]:
# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

jogo = {'Date':[],'Time':[],'Country':[],'League':[],'Home':[],'Away':[],'Odds_H':[],'Odds_D':[],'Odds_A':[],'Odd_BTTS_Yes':[],'Odd_BTTS_No':[]}

In [ ]:
# Pegando as Informacoes Básicas do Jogo
for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/match-summary')
    
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
    except:
        pass

# Match Odds
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/1x2-odds/full-time')
        time.sleep(2)
        
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                Odds_H = linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text
                Odds_D = linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text 
                Odds_A = linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text
    except:
        pass

# Ambas Marcam (BTTS)
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/both-teams-to-score/full-time')
        time.sleep(2)
    
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                BTTS_Yes = linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text
                BTTS_No = linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text
    except:
        pass    
    
    print(Date,Time,Country,League,Home,Away,Odds_H,Odds_D,Odds_A,BTTS_Yes,BTTS_No) 

    jogo['Date'].append(Date)
    jogo['Time'].append(Time)
    jogo['Country'].append(Country)
    jogo['League'].append(League)
    jogo['Home'].append(Home)
    jogo['Away'].append(Away)
    jogo['Odds_H'].append(Odds_H)
    jogo['Odds_D'].append(Odds_D)
    jogo['Odds_A'].append(Odds_A)
    jogo['Odd_BTTS_Yes'].append(BTTS_Yes)
    jogo['Odd_BTTS_No'].append(BTTS_No)

  0%|          | 1/302 [00:06<31:26,  6.27s/it]

08.10.2022 16:30 ENGLAND PREMIER LEAGUE - ROUND 10 Brighton Tottenham 2.50 3.30 2.80 1.75 2.00


  1%|          | 2/302 [00:11<29:20,  5.87s/it]

08.10.2022 19:00 FRANCE LIGUE 1 - ROUND 10 Reims Paris SG 8.00 6.00 1.30 1.80 1.95


  1%|          | 3/302 [00:17<27:40,  5.55s/it]

...

08.10.2022 08:00 WORLD FRIENDLY INTERNATIONAL WOMEN France U23 W Finland U23 W 1.53 4.00 6.00 2.00 1.72


100%|█████████▉| 301/302 [27:17<00:05,  5.21s/it]

08.10.2022 16:30 WORLD FRIENDLY INTERNATIONAL WOMEN Ecuador W Panama W 1.53 4.00 6.00 2.00 1.72


100%|██████████| 302/302 [27:22<00:00,  5.44s/it]

08.10.2022 22:00 WORLD FRIENDLY INTERNATIONAL WOMEN Colombia W Paraguay W 1.53 4.00 6.00 2.00 1.72


In [ ]:
df = pd.DataFrame(jogo)
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df

,Date,Time,Country,League,Home,Away,Odds_H,Odds_D,Odds_A,Odd_BTTS_Yes,Odd_BTTS_No
Nº,,,,,,,,,,,
1,08.10.2022,16:30,ENGLAND,PREMIER LEAGUE - ROUND 10,Brighton,Tottenham,2.50,3.30,2.80,1.75,2.00
2,08.10.2022,19:00,FRANCE,LIGUE 1 - ROUND 10,Reims,Paris SG,8.00,6.00,1.30,1.80,1.95
3,08.10.2022,16:30,GERMANY,BUNDESLIGA - ROUND 9,Dortmund,Bayern Munich,4.00,4.50,1.72,1.44,2.62
4,08.10.2022,18:45,ITALY,SERIE A - ROUND 9,Bologna,Sampdoria,2.15,3.30,3.50,1.80,1.95
5,08.10.2022,16:45,NETHERLANDS,EREDIVISIE - ROUND 9,Sparta Rotterdam,FC Emmen,2.10,3.30,3.60,1.95,1.80
...,...,...,...,...,...,...,...,...,...,...,...
298,08.10.2022,14:00,WORLD,CLUB FRIENDLY,Marko (Gre),Panathinaikos B (Gre),1.53,4.00,6.00,2.00,1.72
299,08.10.2022,14:30,WORLD,CLUB FRIENDLY,Fostiras (Gre),Kalamata (Gre),1.53,4.00,6.00,2.00,1.72
300,08.10.2022,08:00,WORLD,FRIENDLY INTERNATIONAL WOMEN,France U23 W,Finland U23 W,1.53,4.00,6.00,2.00,1.72


In [ ]:
# Gerando os dados para excel
from datetime import date, datetime
data = datetime.today()
hoje = data.strftime('%d-%m-%Y')
nome = 'Primeira_Parte.xlsx'
df.to_excel(f'{hoje}_{nome}', index = False)